In [23]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

df = pd.read_csv("parksdata.csv")
df.head()

,id,name,type,maintenance_responsibility,image,neighborhood,council_district,ward,tract,public_works_division,pli_division,police_zone,fire_zone,latitude,longitude
0,2111959181,Able Long Park,Park,Parks - Southern,https://tools.wprdc.org/images/pittsburgh/park...,Beechview,4,19,42003192000,5,19,6,4-28,40.408118,-80.028197
1,1109513049,Albert Turk Graham Park,Park,Parks - Schenley,https://tools.wprdc.org/images/pittsburgh/park...,Crawford-Roberts,6,3,42003030500,3,3,2,2-1,40.440507,-79.984018
2,1544096765,Alcoa Park,Park,Parks - Northern,NaN,North Shore,1,22,42003563200,6,22,1,1-20,40.447481,-80.001017
3,1212681573,Allegheny Commons,Park,Parks - Northern,NaN,Allegheny Center,1,22,42003562700,1,22,1,1-7,40.453126,-80.009158
4,954863286,Allegheny Landing Park,Park,Parks - Northern,NaN,North Shore,1,22,42003563200,6,22,1,1-20,40.447184,-80.002907


In [24]:
df[["name", "type", "neighborhood"]]


,name,type,neighborhood
0,Able Long Park,Park,Beechview
1,Albert Turk Graham Park,Park,Crawford-Roberts
2,Alcoa Park,Park,North Shore
3,Allegheny Commons,Park,Allegheny Center
4,Allegheny Landing Park,Park,North Shore
...,...,...,...
204,Woods Run Park,Park,Marshall-Shadeland
205,Young Field Park,Park,Marshall-Shadeland
206,Zulema Park,Park,Central Oakland
207,Liberty Green Park,Park,East Liberty


In [25]:
parks_only_df = df[df['type'] == 'Park']

neighborhood_counts = parks_only_df['neighborhood'].value_counts()

top_ten_neighborhoods = neighborhood_counts.head(10)
top_ten_neighborhoods

Beechview             6
South Side Slopes     6
Troy Hill             5
Hazelwood             5
Mount Washington      5
Elliott               4
Sheraden              4
Marshall-Shadeland    4
East Liberty          4
Brighton Heights      3
Name: neighborhood, dtype: int64

In [26]:
df2 = pd.read_csv("arrests.csv")
df2.head()

,PK,CCR,AGE,GENDER,RACE,ARRESTTIME,ARRESTLOCATION,OFFENSES,INCIDENTLOCATION,INCIDENTNEIGHBORHOOD,INCIDENTZONE,INCIDENTTRACT,COUNCIL_DISTRICT,PUBLIC_WORKS_DIVISION,X,Y
0,1975272,16158872,42.0,F,B,2016-08-24T12:20:00,"4700 Block Centre AV Pittsburgh, PA 15213",3929 Retail Theft.,"4700 Block Centre AV Pittsburgh, PA 15213",Bloomfield,5,804.0,8.0,2.0,-79.949277,40.452551
1,1974456,16144120,31.0,M,W,2016-08-03T14:55:00,"4200 Block Steubenville PKE Pittsburgh, PA 15205",13(a)(16) Possession of Controlled Substance,"4200 Block Steubenville PKE Pittsburgh, PA 15205",Outside City,OSC,5599.0,NaN,NaN,-80.088018,40.440136
2,1974466,16144165,63.0,F,B,2016-08-03T16:45:00,"900 Block Freeport RD Fox Chapel, PA 15238",3929 Retail Theft.,"900 Block Freeport RD Fox Chapel, PA 15238",Westwood,5,2811.0,9.0,2.0,-79.891803,40.486625
3,1974550,16145257,25.0,F,W,2016-08-05T02:36:00,"Foreland ST & Cedar AV Pittsburgh, PA 15212",5503 Disorderly Conduct. / 5505 Public Drunken...,"Foreland ST & Cedar AV Pittsburgh, PA 15212",East Allegheny,1,2304.0,1.0,1.0,-80.001939,40.454080
4,1974596,16145962,25.0,M,B,2016-08-06T02:00:00,"900 Block Woodlow ST Pittsburgh, PA 15205",2702 Aggravated Assault. / 2705 Recklessy Enda...,"900 Block Woodlow ST Pittsburgh, PA 15205",Crafton Heights,5,2814.0,2.0,5.0,-80.052204,40.445900


In [28]:
df2[["ARRESTLOCATION", "INCIDENTNEIGHBORHOOD"]]

,ARRESTLOCATION,INCIDENTNEIGHBORHOOD
0,"4700 Block Centre AV Pittsburgh, PA 15213",Bloomfield
1,"4200 Block Steubenville PKE Pittsburgh, PA 15205",Outside City
2,"900 Block Freeport RD Fox Chapel, PA 15238",Westwood
3,"Foreland ST & Cedar AV Pittsburgh, PA 15212",East Allegheny
4,"900 Block Woodlow ST Pittsburgh, PA 15205",Crafton Heights
...,...,...
61371,"900 Block 2nd AV Pittsburgh, PA 15219",Larimer
61372,"2000 Block Centre AV Pittsburgh, PA 15219",Crawford-Roberts
61373,"East Liberty BL & Mellon ST Pittsburgh, PA 15206",East Liberty
61374,"S Bouquet ST & Dawson ST Pittsburgh, PA 15213",Central Oakland


In [45]:
neighborhoods1 = set(df['neighborhood'])
neighborhoods2 = set(df2['INCIDENTNEIGHBORHOOD'])

common_neighborhoods = neighborhoods1.intersection(neighborhoods2)

print(common_neighborhoods)

#new_arrests_data = df2[df2['INCIDENTNEIGHBORHOOD']]

new_arrests = df2[df2['INCIDENTNEIGHBORHOOD'].isin(common_neighborhoods)]

{'Stanton Heights', 'South Side Flats', 'Arlington', 'Elliott', 'Troy Hill', 'Marshall-Shadeland', 'Windgap', 'Hazelwood', 'Fairywood', 'Squirrel Hill South', 'Mount Washington', 'Carrick', 'Point Breeze', 'Perry South', 'Esplen', 'North Oakland', 'Highland Park', 'Greenfield', 'Lincoln-Lemington-Belmar', 'East Hills', 'Bluff', 'Perry North', 'Fineview', 'Lincoln Place', 'Point Breeze North', 'Oakwood', 'North Shore', 'Strip District', 'East Liberty', 'Bedford Dwellings', 'Polish Hill', 'South Oakland', 'Squirrel Hill North', 'Central Lawrenceville', 'Regent Square', 'Westwood', 'Sheraden', 'Central Northside', 'Homewood South', 'Manchester', 'Homewood North', 'Spring Hill-City View', 'Swisshelm Park', 'Bloomfield', 'West End', 'Allentown', 'Central Oakland', 'Brighton Heights', 'South Side Slopes', 'Duquesne Heights', 'Allegheny Center', 'Banksville', 'Central Business District', 'Crafton Heights', 'Garfield', 'Terrace Village', 'New Homestead', 'Beechview', 'Upper Lawrenceville', 'Ho

In [50]:
counted_arrests = new_arrests['INCIDENTNEIGHBORHOOD'].value_counts()
counted_arrests.head(10)
top_ten_neighborhoods = counted_arrests.nsmallest(10)
top_ten_neighborhoods

Central Northside     23
Regent Square         35
New Homestead         39
Swisshelm Park        40
East Carnegie         48
Oakwood               79
Hays                 106
Fairywood            112
Windgap              115
Polish Hill          153
Name: INCIDENTNEIGHBORHOOD, dtype: int64